<a href="https://colab.research.google.com/github/JoaoVlopess/Graph-Algorithms-Projects/blob/main/C%C3%B3pia_de_Jantar_dos_filosofos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import threading
import time
import random

# --- CONFIGURAÇÕES GLOBAIS ---
NUM_FILOSOFOS = 5
NUM_RODADAS = 10
# Recurso de sincronização para output
print_lock = threading.Lock()
# Lista para referência dos objetos Filósofo
filosofos_list = []

def log_print(message):
    """ Imprime uma mensagem com timestamp de forma thread-safe. """
    with print_lock:
        timestamp = time.strftime('%H:%M:%S', time.localtime())
        print(f"{timestamp} {message}")
        time.sleep(0.4)


class Filosofo(threading.Thread):
    """ Representa o filósofo, que é passivo e espera o sinal do Coordenador. """
    def __init__(self, id_filosofo, garfo_esquerda, garfo_direita):
        super().__init__()
        self.id = id_filosofo
        self.garfo_esquerda = garfo_esquerda
        self.garfo_direita = garfo_direita
        self.contador_comidas_total = 0

        # Evento que o Coordenador usa para sinalizar permissão de comer
        self.pode_comer_event = threading.Event()
        # Evento que o Filósofo usa para sinalizar ao Coordenador que terminou
        self.terminou_de_comer_event = threading.Event()
        self.deve_parar = False

    def run(self):
        """ Ciclo de vida: pensar, esperar e comer quando autorizado. """
        self.pensar(0)

        while True:
            # Bloqueia até receber o sinal do Coordenador
            self.pode_comer_event.wait()

            if self.deve_parar:
                break

            self.pegar_garfos()
            self.comer()
            self.devolver_garfos()
            # Sinaliza ao Coordenador que a refeição terminou
            self.terminou_de_comer_event.set()

            self.pensar("...")

        log_print(f"Filósofo {self.id + 1} terminou seu jantar.")

    def pensar(self, rodada_num):
        tempo_pensando = random.uniform(1.0, 1.5)
        log_print(f"Filósofo {self.id + 1} (Ciclo {rodada_num}) está PENSANDO.")
        time.sleep(tempo_pensando)

    def pegar_garfos(self):
        """ Adquire ambos os locks dos garfos sequencialmente. """
        log_print(f"Filósofo {self.id + 1} está com FOME e pega o garfo ESQUERDO.")
        self.garfo_esquerda.acquire()
        log_print(f"Filósofo {self.id + 1} pegou o garfo ESQUERDO (pegando o direito).")

        time.sleep(0.5)

        log_print(f"Filósofo {self.id + 1} pega o garfo DIREITO.")
        self.garfo_direita.acquire()
        log_print(f"--> Filósofo {self.id + 1} pegou AMBOS. Está APTO PARA COMER.")

    def comer(self):
        """ Simula o tempo de consumo do recurso. """
        tempo_comendo = random.uniform(1.0, 1.5)
        log_print(f"Filósofo {self.id + 1} está COMENDO.")
        self.contador_comidas_total += 1
        time.sleep(tempo_comendo)

    def devolver_garfos(self):
        """ Libera os locks dos garfos para uso de outros filósofos. """
        self.garfo_esquerda.release()
        self.garfo_direita.release()
        log_print(f"Filósofo {self.id + 1} DEVOLVEU os garfos.")


class Coordenador(threading.Thread):
    """
    Controla os ciclos de simulação, implementando a política de acesso
    (alternância F_i e F_{i+2}) para prevenir deadlock e starvation.
    """
    def __init__(self, filosofos, num_rodadas):
        super().__init__()
        self.filosofos = filosofos
        self.num_rodadas = num_rodadas

    def run(self):
        log_print("Sistema de coordenação de ciclos iniciado.")

        for i in range(self.num_rodadas):
            rodada_num = i + 1
            with print_lock:
                print("\n" + "="*50)
                print(f"--- INÍCIO DO CICLO {rodada_num} ---")

            # Determina quais filósofos podem comer no ciclo atual (i e i+2)
            id1 = i % NUM_FILOSOFOS
            id2 = (i + 2) % NUM_FILOSOFOS

            f1 = self.filosofos[id1]
            f2 = self.filosofos[id2]

            log_print(f"(Ciclo {rodada_num}) Autorizando Filósofos {f1.id + 1} e {f2.id + 1} a comer.")

            # Reseta os eventos de término
            f1.terminou_de_comer_event.clear()
            f2.terminou_de_comer_event.clear()

            # Acorda os filósofos autorizados
            f1.pode_comer_event.set()
            f2.pode_comer_event.set()

            # Espera que ambos os filósofos terminem de comer
            f1.terminou_de_comer_event.wait()
            f2.terminou_de_comer_event.wait()

            # Reseta os eventos de permissão para o próximo ciclo
            f1.pode_comer_event.clear()
            f2.pode_comer_event.clear()

            with print_lock:
                print("="*50)
                print(f"--- FIM DO CICLO {rodada_num} ---")
                print(f"   Filósofo {f1.id + 1} COMEU (Total: {f1.contador_comidas_total})")
                print(f"   Filósofo {f2.id + 1} COMEU (Total: {f2.contador_comidas_total})")
                # Exibe o status dos que não comeram neste ciclo
                for f in self.filosofos:
                    if f.id != id1 and f.id != id2:
                        print(f"   Filósofo {f.id + 1} não comeu (Total: {f.contador_comidas_total})")
                print("="*50 + "\n")
                time.sleep(1)

        log_print("Todos os ciclos terminaram. Mandando filósofos pararem...")

        # Sinaliza a todos para saírem do loop principal
        for f in self.filosofos:
            f.deve_parar = True
            f.pode_comer_event.set()


def main():
    global filosofos_list

    log_print(f"O Jantar dos Filósofos vai começar! ({NUM_FILOSOFOS} filósofos, {NUM_RODADAS} ciclos)")

    # 1. Cria os locks (garfos)
    garfos = [threading.Lock() for _ in range(NUM_FILOSOFOS)]

    # 2. Cria os objetos Filósofo, atribuindo os garfos esquerdo e direito
    for i in range(NUM_FILOSOFOS):
        garfo_esquerda = garfos[i]
        garfo_direita = garfos[(i + 1) % NUM_FILOSOFOS]

        filosofo = Filosofo(i, garfo_esquerda, garfo_direita)
        filosofos_list.append(filosofo)

    # 3. Cria e inicia as threads
    coordenador = Coordenador(filosofos_list, NUM_RODADAS)

    for filosofo in filosofos_list:
        filosofo.start()

    coordenador.start()

    # 4. Aguarda o término de todas as threads
    for filosofo in filosofos_list:
        filosofo.join()

    coordenador.join()

    # 5. Resumo da simulação
    log_print("-----------------------------------------")
    log_print("---       Resumo FINAL do Jantar      ---")
    log_print("-----------------------------------------")
    for filosofo in filosofos_list:
        log_print(f"Filósofo {filosofo.id + 1} comeu um total de {filosofo.contador_comidas_total} vezes.")
    log_print("-----------------------------------------")
    log_print("O Jantar dos Filósofos terminou!")

if __name__ == "__main__":
    main()

13:22:09 O Jantar dos Filósofos vai começar! (5 filósofos, 10 ciclos)
13:22:09 Filósofo 1 (Ciclo 0) está PENSANDO.
13:22:09 Filósofo 2 (Ciclo 0) está PENSANDO.
13:22:10 Filósofo 3 (Ciclo 0) está PENSANDO.
13:22:10 Filósofo 4 (Ciclo 0) está PENSANDO.
13:22:11 Filósofo 5 (Ciclo 0) está PENSANDO.
13:22:11 Sistema de coordenação de ciclos iniciado.

--- INÍCIO DO CICLO 1 ---
13:22:11 (Ciclo 1) Autorizando Filósofos 1 e 3 a comer.
13:22:12 Filósofo 3 está com FOME e pega o garfo ESQUERDO.
13:22:12 Filósofo 3 pegou o garfo ESQUERDO (pegando o direito).
13:22:13 Filósofo 1 está com FOME e pega o garfo ESQUERDO.
13:22:13 Filósofo 1 pegou o garfo ESQUERDO (pegando o direito).
13:22:13 Filósofo 3 pega o garfo DIREITO.
13:22:14 --> Filósofo 3 pegou AMBOS. Está APTO PARA COMER.
13:22:14 Filósofo 3 está COMENDO.
13:22:15 Filósofo 1 pega o garfo DIREITO.
13:22:15 --> Filósofo 1 pegou AMBOS. Está APTO PARA COMER.
13:22:15 Filósofo 1 está COMENDO.
13:22:16 Filósofo 3 DEVOLVEU os garfos.
13:22:16 Filós